# Lab 6: Transfer Learning

In the lab, we are going to train an object detection CNN 
using Transfer Learning. The idea is to start from your 
lab 5 network and re-use it to speed up the training 
on a new dataset.

In [ ]:
# Import the necessary modules

import keras
from keras import datasets
from keras.layers import Input, Dense, Flatten, Dropout, Activation 
from keras.layers import PReLU, LeakyReLU, Conv2D, Lambda, MaxPooling2D
from keras.regularizers import l2

from keras.layers import BatchNormalization

from keras.models import model_from_json

from IPython.display import clear_output

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator

import pickle
import sklearn as skl

from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

In [ ]:
# Define some useful functions
class PlotLossAccuracy(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.acc = []
        self.losses = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(int(self.i))
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        
        self.i += 1
        
        clear_output(wait=True)
        plt.figure(figsize=(16, 6))
        plt.plot([1, 2])
        plt.subplot(121) 
        plt.plot(self.x, self.losses, label="train loss")
        plt.plot(self.x, self.val_losses, label="validation loss")
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.title('Model Loss')
        plt.legend()
        plt.subplot(122)         
        plt.plot(self.x, self.acc, label="training accuracy")
        plt.plot(self.x, self.val_acc, label="validation accuracy")
        plt.legend()
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.title('Model Accuracy')
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.show();
        
def save_model_to_disk():    
    # save model and weights (don't change the filenames)
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to model.json and weights to model.h5")

# Loading the new dataset (cifar100)

We are going to use the 10 first classes of the cifar100 dataset.
https://www.cs.toronto.edu/~kriz/cifar.html

> This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).


In [ ]:
!curl --create-dirs -o /home/tcd/data/cifar100-dataset.pkl http://mindfreeze.ml/~mindfreeze/cifar100-dataset.pkl


In [ ]:
# Loading new dataset cifar100

print('loading the dataset...')

pkl_file = open('/home/tcd/data/cifar100-dataset.pkl', 'rb')
dataset = pickle.load(pkl_file)

print('loaded.')

print('let\'s look at some of the pictures and their ground truth labels:')

plt.figure(figsize=(12,12))
plt.plot([3, 3])

X = dataset['X'].astype('float32')/255
Y = dataset['Y'].astype('float32')
Y = keras.utils.to_categorical(Y)

for i in range(0,9):
    # pictures are 32x32x3 (width=32, height=32, 3 colour channels)
    pic = X[i]

    # Y[i] returns an array of zeros and with Y[i][classid] = 1
    # for instance  Y[i] = [ 0 0 0 0 0 1 0 0 0 0] => classid=5 
    #          and  Y[i] = [ 1 0 0 0 0 0 0 0 0 0] => classid=0
    # we can get the classid by using the argmax function on the vector Y[i]
    classid = Y[i].argmax(-1)

    # getting back the name of the label for that classid
    classname = dataset['labels'][classid]
    plt.subplot(3,3,i+1)
    plt.imshow(pic)
    plt.title('label: {}'.format(classname))


In [ ]:
# training / validation split
X_train, X_validation, Y_train, Y_validation = skl.model_selection.train_test_split(X, Y, test_size=.1, random_state=0)

## Loading the Base Network

Our starting point will be to take your CNN network from Lab 5 
and use it for building visual features for the images of our new dataset.
So let's start with re-loading your network. 

You don't need to retrain your network but you can if you want. If you do, note that Also, 
note that we probably don't need a very long training. The idea of this lab 
is only to show that you can get a good kick start from lab 5's network.


In [ ]:
# Here we assume that you have already successfully completed Lab 5 
# and trained a CNN on cifar10

# load json and create base_model
with open('../lab-05/model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
base_model = model_from_json(loaded_model_json)
# load weights into base_model
base_model.load_weights("../lab-05/model.h5")


In [ ]:
# print the model summary
base_model.summary()

# In the summary, take note of the Layer names in the first column
# eg. conv2d_6 (Conv2D) means that 'conv2d_6' will be the name for 
# that layer

# In preparation for the next step, You also need to check that 
# shape of the Flatten layer is small enough. Typically we want something
# like (None, 2048). You don't want (None, 40000).

# Part 1: Using the Base Network to Build Visual Features

In this first part we are going to directly use the base model to pre-compute 
visual features on this new dataset. For this, we are going to compute the predictions 
for the intermediate `Flatten` layer.

In [ ]:
# change the `flatten_2` name to the corresponding name from your network

features_model = keras.Model(inputs=base_model.input,
                    outputs=base_model.get_layer('flatten_2').output)
train_features = features_model.predict(X_train)
validation_features = features_model.predict(X_validation)

# checking that everything is ok, you should get a tensor of size 4500 x Flatten Size
# eg. (4500, 2048)
print(train_features.shape)


In [ ]:
# now that you have computed the new features for your dataset,
# you need to design a simple classification network.
# For good measure, use one with BatchNormalization, dropout, etc. 

inputs = keras.layers.Input(shape=train_features.shape[1:]) 
# TODO: complete the network ....
x = Dense(512, activation = 'relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(256, activation = 'relu')(inputs)
x = BatchNormalization()(x)
#x = Dropout(0.2)(x)

predictions = Dense(10, activation='softmax')(x)

In [ ]:
# now we can compile it and train it.

top_model = keras.models.Model(inputs=inputs, outputs=predictions)

top_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

top_model.summary()

In [ ]:

# TODO: write code to train it ...


# the dataset is very small and the base_model gives us a very good 
# starting point, so we only need a handful of epochs, probably 1 
# or 2 will do.

# top_model.fit( ... )

num_epochs = 4

# Create an instance of our callback functions class, to plot our loss function and accuracy with each epoch.
pltCallBack = PlotLossAccuracy()

# Run the training.
top_model.fit(train_features, Y_train,
          batch_size=4096, epochs=num_epochs, 
          validation_data=(validation_features, Y_validation), 
          callbacks=[pltCallBack])


# Part 2: Fine-Tuning the Base Network

In this part, instead of pre-computing the visual features, 
we are going to load the base model, freeze the weights for the convolutional layers and allow the optimizer to fine-tune the classification dense layers. This is done by setting the flag
`trainable` to `True` or `False` in the network layers.


In [ ]:
model = keras.models.clone_model(base_model)

# set the all the layers in model to trainable = False

for layer in model.layers:
    layer.trainable = False

    
model.layers[14].trainable = True
model.layers[16].trainable = True
model.layers[18].trainable = True
model.layers[19].trainable = True

for layer in model.layers:
   print(layer, layer.trainable)

# TODO: then set all the layers after the Flatten layer to be trainable    

In [ ]:
# TODO compile model 

# model.compile(...)
opt = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
# model.summary()
model.summary()
# loading the pre-trained weights from your lab-05 network
model.load_weights("../lab-05/model.h5")


In [ ]:
# TODO train 'model' on (X_train, Y_train) for 10 or so epochs  
# What you should be able to observe is that the training is a bit slower than
# in the previous case. This is partly because we haven't re-initialised the weights.
num_epochs = 20

# Create an instance of our callback functions class, to plot our loss function and accuracy with each epoch.
pltCallBack = PlotLossAccuracy()

# Run the training.
model.fit(X_train, Y_train,
          batch_size=4096, epochs=num_epochs, 
          validation_data=(X_validation, Y_validation), 
          callbacks=[pltCallBack])


In [ ]:
# write model to model.json and weights to model.h5 for submission

save_model_to_disk()

In [ ]:
# In the terminal, add model.json and model.h5 to git and submit the lab
# Do the following: 
#  git add lab-06/model.json lab-06/model.h5
#  git commit -m "Added NN model."
#  git push
#  submit-lab 6

In [ ]:
# In the terminal, add model.json and model.h5 to git and submit the lab
# Do the following: 
#  git add lab-06/model.json lab-06/model.h5
#  git commit -m "Added NN model."
#  git push
#  submit-lab 6